In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.stem.porter import *
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.preprocessing import StandardScaler
# ML Libraries
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

# Global Parameters
stop_words = set(stopwords.words('english'))

In [ ]:
df = pd.read_csv('train.csv', )
test = pd.read_csv('test.csv')


In [ ]:
df.head(30)

In [ ]:
df.tail()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df['message'] = df['message'].fillna(0)

In [ ]:
#Remove URL
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df['message'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
#Remove punctuation
df['message']  = df['message'] .str.lower()
def remove_punctuation(message):
    return ''.join([l for l in message if l not in string.punctuation])

In [ ]:
df['message'] = df['message'].apply(remove_punctuation)

In [ ]:
#Tokenize Data
tokeniser = TreebankWordTokenizer()
df['message'] = df['message'].apply(tokeniser.tokenize)

In [ ]:
#Stemming
stemmer = PorterStemmer()
def df_stemmer(words, stemmer):
    return [stemmer.stem(word) for word in words]
df['message'] = df['message'].apply(df_stemmer, args=(stemmer, ))

In [ ]:
#Lemmatization
lemmatizer = WordNetLemmatizer()
def df_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]    

In [ ]:
df['message'] = df['message'].apply(df_lemma, args=(lemmatizer, ))

In [ ]:
#Remove Stopwords

def remove_stop_words(tokens):    
    return [t for t in tokens if t not in stopwords.words('english')]

In [ ]:
df['message']=df['message'].apply(remove_stop_words)

In [ ]:
#Feature Selection Splitting out Variables
y =df['sentiment']
X = df['message']

bow_vect = CountVectorizer(max_features=1000, stop_words='english')
X_vect=bow_vect.fit_transform(df[['message']])

trans = StandardScaler(with_mean=False)
X_vect = trans.fit_transform(X_vect)

In [ ]:
#Training Model
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2, random_state=42)

In [ ]:
X_vect

In [ ]:
#Model training
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

In [ ]:
#Test Set

testx = test['message']
test_vect = bow_vect.transform(testx)

In [ ]:
#Predications
y_pred = rfc.predict(test_vect)

In [ ]:
#Test
test['sentiment'] = y_pred

In [ ]:
test.head()

In [ ]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)